In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')
import os
root = "/content/gdrive/My Drive/Colab Notebooks/a/vit"
# if not os.path.exists(root):
#     os.makedirs(root)
os.chdir(root)

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [2]:
import os
import sys
if sys.version_info[0] == 2:
    import cPickle as pickle
else:
    import pickle

import torch
import torch.nn as nn
import numpy as np
import torch.optim as optim
from tqdm import tqdm

import save as SAVE
import data as DATA
import models as MODELS

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
device


'cuda:0'

In [3]:
device = 'cpu'
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
device

'cuda:0'

In [4]:
TEST = True
OUTPUT_ROOT = 'output'
DATA_DIR = 'data'

In [5]:
BACKBONE_TYPE = 'resnet' #'mae','resnet'
TASK = 'classification' #'classification', 'segmentation', 'depth'
MODE = 'last' #'last','e2e'
DATA_NAME = 'cifer10' #cifer10,nyudepth

SAVE.init(OUTPUT_ROOT,BACKBONE_TYPE,TASK,MODE,DATA_NAME,TEST)
DATA.init(DATA_DIR)

normalize_data = {
    'resnet':True,
    'mae':False
}[BACKBONE_TYPE]
preprocessor = MODELS.get_preprocessor(BACKBONE_TYPE)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/convnext/feature_extraction_convnext.py:28: FutureWarning: The class ConvNextFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ConvNextImageProcessor instead.
  warnings.warn(


In [6]:
N_WORKERS = 0 #0 if parallelism fails
BS = 128
EPOCH = 10
LR = 1e-3

if TEST:
    N_WORKERS = 0 #0 if parallelism fails
    BS = 5
    EPOCH = 6
    head_n = 15

# Model

In [7]:
def count_model_size(model):
    size = 0
    for name, param in model.named_parameters():
        size += param.numel()
    return size

In [8]:
model = MODELS.get_model(BACKBONE_TYPE,TASK,MODE)
# preprocessor = MODELS.get_preprocessor(BACKBONE_TYPE)
model = model.to(device)
count_model_size(model)

UnboundLocalError: local variable 'train' referenced before assignment

# Dataset

In [ ]:
get_set = DATA.get_set(DATA_NAME)
transform = DATA.get_transform(DATA_NAME,BACKBONE_TYPE,train=True)
trainset = get_set(root=DATA_DIR, train=True, download=True, transform=transform)
transform = DATA.get_transform(DATA_NAME,BACKBONE_TYPE,train=False)
testset  = get_set(root=DATA_DIR, train=False, download=True, transform=transform)

# testset = DATA.head(testset,head_n)
if TEST:
    trainset = DATA.head(trainset,head_n)
    testset = DATA.head(testset,head_n)


trainloader = torch.utils.data.DataLoader(trainset, batch_size=BS,
                                          shuffle=True, num_workers=N_WORKERS)


testloader = torch.utils.data.DataLoader(testset, batch_size=BS,
                                         shuffle=False, num_workers=N_WORKERS)



# Train

In [ ]:
criterion = nn.CrossEntropyLoss()
if MODE == 'last':
    params = model.added.parameters()
if MODE == 'e2e':
    params = model.parameters()

optimizer = optim.AdamW(params, lr=LR)
num_epochs = EPOCH
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=8, gamma=0.8)

In [ ]:
def evaluate_model(loader):
    num_correct = 0
    total = 0
    eval_loss = 0.0
    with torch.no_grad():
        for i, data in tqdm(enumerate(loader, 0), total=len(loader)):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)

            inputs = preprocessor(inputs, return_tensors="pt").to(device)

            with torch.no_grad():
                outputs = model(**inputs).logits

            loss = criterion(outputs, labels).item()
            eval_loss += loss
            prediction = outputs.argmax(dim=-1)

            total += prediction.size(0)
            num_correct += (prediction == labels).sum().item()
    return eval_loss/len(loader), num_correct / total


In [ ]:
train_losses = []
test_losses = []
test_acc = []

# Train
eval_freq = 3
test_loss_min = np.Inf
for epoch in range(num_epochs):  # loop over the dataset multiple times
    model.train()
    train_loss_buffer = []
    for i, data in tqdm(enumerate(trainloader, 0), total=len(trainloader)):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        # inputs = preprocessor(inputs)
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        inputs = preprocessor(inputs, return_tensors="pt").to(device)

        outputs = model(**inputs).logits

        loss = criterion(outputs, labels)
        train_loss_buffer.append(loss.item())
        loss.backward()
        optimizer.step()

    mean_train_loss = np.mean(train_loss_buffer)
    print(f'[{epoch}] train loss: {mean_train_loss}')
    train_losses.append(mean_train_loss)
    train_loss_buffer.clear()

    # Update evaluation on test set
    if epoch % eval_freq == 0:
        print('update evaluation')
        model.eval()
        test_loss, correctness_test = evaluate_model(testloader)

    test_losses.append(test_loss)
    test_acc.append(correctness_test)

    print(f'''
            [{epoch}]
            train loss: {train_losses[-1]:.3f},
            valid_loss: {test_loss:.3f},
            acc: {correctness_test*100}%,
            lr={optimizer.param_groups[0]["lr"]}
        ''')

    # save model if validation loss has decreased
    if test_loss <= test_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        test_loss_min,
        test_loss))

        SAVE.save_model(model)
        test_loss_min = test_loss
    scheduler.step()

SAVE.save_log(train_losses,test_losses,test_acc)


print('Finished Training')

In [ ]:
print(train_losses)

In [ ]:
model.eval()
evaluate_model(testloader)